In [ ]:
!nvidia-smi

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="bVCHIdUmWTMSr6ETOu3O")
project = rf.workspace("computer-vision-yqyos").project("grape-segmentation-yedek-kendi-verim")
version = project.version(27)
dataset = version.download("yolov11")

In [ ]:
import torch
import torch.nn as nn
from ultralytics import YOLO
from ultralytics.nn.modules.block import C2f

# ==================== CBAM Modülleri ====================
class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=8):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.fc = nn.Sequential(
            nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False),
            nn.ReLU(),
            nn.Conv2d(in_planes // ratio, in_planes,    1, bias=False),
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc(self.avg_pool(x))
        max_out = self.fc(self.max_pool(x))
        return self.sigmoid(avg_out + max_out) * x

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super().__init__()
        padding = kernel_size // 2
        self.conv = nn.Conv2d(2, 1, kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = x.mean(dim=1, keepdim=True)
        max_out, _ = x.max(dim=1, keepdim=True)
        return self.sigmoid(self.conv(torch.cat([avg_out, max_out], dim=1))) * x

class CBAM(nn.Module):
    def __init__(self, channels, ratio=8, kernel_size=7):
        super().__init__()
        self.channel_attention = ChannelAttention(channels, ratio)
        self.spatial_attention = SpatialAttention(kernel_size)

    def forward(self, x):
        x = self.channel_attention(x)
        x = self.spatial_attention(x)
        # Eğitim sırasında her forward’da bu mesajı göreceksin
        print("✅ CBAM forward tetiklendi")
        return x

# ==================== CBAM Entegrasyonu ====================
def integrate_cbam_to_yolo(yolo_model):
    """
    Model katmanları arasında C2f bloklarını bulup sonuna CBAM ekler,
    doğru kanal sayısını layer.cv2.conv.out_channels ile alır.
    """
    layers = yolo_model.model.model  # ultralytics iç katman listesi
    for idx, layer in enumerate(layers):
        if isinstance(layer, C2f):
            # C2f içindeki cv2, bir Conv modülü; onun .conv.out_channels ile alıyoruz
            ch = layer.cv2.conv.out_channels
            layers[idx] = nn.Sequential(layer, CBAM(ch))
            print(f"✅ CBAM eklendi: C2f @ idx {idx}, channels={ch}")
    return yolo_model

# ==================== Ana Fonksiyon ====================
def main():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # Önceden eğitilmiş segmentation modelini yükle
    yolo = YOLO('yolo11m-seg.pt')
    yolo.model.to(device)

    # Tüm C2f bloklarına CBAM ekle
    yolo = integrate_cbam_to_yolo(yolo)

    # Eğitim
    yolo.train(
        data='/teamspace/studios/this_studio/Grape-Segmentation---yedek---kendi-verim-27/data.yaml',
        epochs=75,
        imgsz=640,
        batch=16,
        project='YOLOv11m_CBAM_InstSeg',
        name='exp1',
        device=device,
        exist_ok=True,
        optimizer='AdamW',
        lr0=0.001
    )

if __name__ == '__main__':
    main()


✅ CBAM eklendi: C2f @ idx 2, channels=256
✅ CBAM eklendi: C2f @ idx 4, channels=512
✅ CBAM eklendi: C2f @ idx 6, channels=512
✅ CBAM eklendi: C2f @ idx 8, channels=512
✅ CBAM eklendi: C2f @ idx 13, channels=512
✅ CBAM eklendi: C2f @ idx 16, channels=256
✅ CBAM eklendi: C2f @ idx 19, channels=512
✅ CBAM eklendi: C2f @ idx 22, channels=512
Ultralytics 8.3.139 🚀 Python-3.10.10 torch-2.7.0+cu128 CUDA:0 (NVIDIA L4, 22491MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/teamspace/studios/this_studio/Grape-Segmentation---yedek---kendi-verim-27/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=75, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0

YOLO11m-seg summary: 253 layers, 22,362,300 parameters, 22,362,284 gradients, 123.6 GFLOPs

Transferred 273/711 items from pretrained weights
Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1394.8±347.1 MB/s, size: 55.7 KB)


train: Scanning /teamspace/studios/this_studio/Grape-Segmentation---yedek---kendi-verim-27/train/labels.cache... 5068 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5068/5068 [00:00<?, ?it/s]

train: /teamspace/studios/this_studio/Grape-Segmentation---yedek---kendi-verim-27/train/images/IMG_3172_jpg.rf.f4d311447f4b3e0014d6cbaf0ffcce28.jpg: 1 duplicate labels removed


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 726.3±499.6 MB/s, size: 65.8 KB)


...

      32/75      18.3G     0.8377       1.29     0.7401     0.9939        201        640: 100%|██████████| 317/317 [03:37<00:00,  1.46it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  1.86it/s]

                   all         87        622      0.639      0.672      0.665      0.471      0.654      0.683      0.681      0.452



      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


      33/75      18.4G     0.8294      1.263     0.7218      0.985        252        640:  44%|████▍     | 141/317 [01:37<02:01,  1.45it/s]


KeyboardInterrupt: 